In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import os
import re
import random
import traceback

# 设置基础URL
base_url = "https://www.ricedata.cn/variety/"

# 创建保存数据的目录
if not os.path.exists('rice_data'):
    os.makedirs('rice_data')

# 全局请求头
REQUEST_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': base_url + 'index.htm'
}

def get_province_codes():
    """获取所有省份的特殊代号"""
    try:
        print("正在获取省份列表...")
        response = requests.get(base_url + "index.htm", headers=REQUEST_HEADERS)
        
        # 动态检测编码
        if response.encoding == 'ISO-8859-1':
            # 尝试常见中文编码
            for encoding in ['gbk', 'gb2312', 'utf-8']:
                try:
                    response.encoding = encoding
                    soup = BeautifulSoup(response.text, 'html.parser')
                    # 检查是否有中文内容
                    if '水稻' in soup.text:
                        print(f"使用编码: {encoding}")
                        break
                except:
                    continue
        
        if response.status_code != 200:
            print(f"请求失败，状态码: {response.status_code}")
            return {}
        
        soup = BeautifulSoup(response.text, 'html.parser')
        provinces = {}

        # 查找所有包含省份链接的a标签
        for link in soup.find_all('a', href=True):
            href = link['href']
            if href.startswith("identified/"):
                province_name = link.get_text(strip=True)
                match = re.search(r'identified/(.*?)_', href)
                if match:
                    code = match.group(1)
                    provinces[province_name] = code
                    print(f"找到省份: {province_name} ({code})")
        
        print(f"共找到 {len(provinces)} 个省份/部门")
        return provinces
    except Exception as e:
        print(f"获取省份代号失败: {e}")
        return {}

def get_total_pages(province_code):
    """获取指定省份的总页数 - 优化页码提取逻辑"""
    try:
        first_page_url = f"{base_url}identified/{province_code}_1.htm"
        print(f"获取总页数: {first_page_url}")
        response = requests.get(first_page_url, headers=REQUEST_HEADERS)
        
        # 动态检测编码
        if response.encoding == 'ISO-8859-1':
            for encoding in ['gbk', 'gb2312', 'utf-8']:
                try:
                    response.encoding = encoding
                    soup = BeautifulSoup(response.text, 'html.parser')
                    if '水稻' in soup.text:
                        print(f"使用编码: {encoding}")
                        break
                except:
                    continue
        
        if response.status_code != 200:
            print(f"请求失败，状态码: {response.status_code}")
            return 1
            
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 优化页码提取逻辑
        total_pages = 1
        
        # 1. 尝试查找末页链接
        last_page_link = soup.find('a', attrs={"title": "跳至末页"})
        if not last_page_link:
            last_page_link = soup.find('a', string='>>')
        
        if last_page_link:
            href = last_page_link.get('href', '')
            if href:
                match = re.search(r'_(\d+)\.htm$', href)
                if match:
                    total_pages = int(match.group(1))
                    print(f"从末页链接提取总页数: {total_pages}")
        
        # 2. 如果未找到，尝试从分页文本中提取
        if total_pages == 1:
            pagination = soup.find('div', align='center')
            if pagination:
                # 查找类似 "共122页" 的文本
                page_text = pagination.get_text()
                match = re.search(r'共(\d+)页', page_text)
                if match:
                    total_pages = int(match.group(1))
                    print(f"从分页文本提取总页数: {total_pages}")
        
        # 3. 如果还是未找到，尝试从分页链接中推断
        if total_pages == 1:
            print("尝试从分页元素提取最大页码")
            page_links = soup.select('a[href*="_"]')
            max_page = 1
            for link in page_links:
                href = link.get('href', '')
                if href:
                    match = re.search(r'_(\d+)\.htm$', href)
                    if match:
                        page_num = int(match.group(1))
                        if page_num > max_page:
                            max_page = page_num
            total_pages = max_page
            print(f"推断最大页码为: {total_pages}")
        
        return total_pages
    
    except Exception as e:
        print(f"获取 {province_code} 总页数失败: {e}")
        return 1

def scrape_page(province_code, page):
    """爬取单页数据 - 优化序号列提取"""
    headers = []
    data = []
    
    try:
        page_url = f"{base_url}identified/{province_code}_{page}.htm"
        print(f"正在爬取: {page_url}")
        response = requests.get(page_url, headers=REQUEST_HEADERS)
        
        # 动态检测编码
        if response.encoding == 'ISO-8859-1':
            # 尝试常见中文编码
            for encoding in ['gbk', 'gb2312', 'utf-8']:
                try:
                    response.encoding = encoding
                    soup = BeautifulSoup(response.text, 'html.parser')
                    # 检查是否有中文内容
                    if '水稻' in soup.text:
                        print(f"使用编码: {encoding}")
                        break
                except:
                    continue
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 优化表格定位 - 更通用的表格识别方式
        table = None
        
        # 1. 通过特定样式定位
        table = soup.find('table', style=re.compile(r'font-size:13;'))
        
        # 2. 通过特定宽度定位
        if not table:
            table = soup.find('table', width='1200')
        
        # 3. 通过表格内容定位
        if not table:
            all_tables = soup.find_all('table')
            for t in all_tables:
                table_text = t.get_text(strip=True)
                if '序号' in table_text and '品种名称' in table_text:
                    table = t
                    break
        
        if not table:
            print("未找到目标表格")
            return headers, data
        
        # 打印表格HTML片段
        table_html = str(table)[:500]  # 只打印前500个字符
        print(f"表格HTML片段: {table_html}...")
        
        # 优化表头提取 - 确保包含序号列
        # 查找所有可能包含表头的行
        header_rows = []
        for row in table.find_all('tr'):
            # 检查行是否包含表头关键词
            row_text = row.get_text(strip=True)
            if '序号' in row_text and '品种名称' in row_text:
                header_rows.append(row)
        
        # 如果没有找到明确的表头行，使用前两行作为候选
        if not header_rows:
            header_rows = table.find_all('tr', limit=2)
        
        # 遍历候选行，选择最可能是表头的行
        for row in header_rows:
            header_cells = []
            # 收集所有单元格（包括th和td）
            for cell in row.find_all(['th', 'td']):
                # 提取文本并清理
                cell_text = cell.get_text(strip=True).replace('\n', ' ').replace('\t', ' ').replace('  ', ' ')
                if cell_text:
                    header_cells.append(cell_text)
            
            # 检查是否包含序号列
            if header_cells and '序号' in header_cells[0]:
                headers = header_cells
                print(f"找到有效表头: {headers}")
                # 记录表头行的位置
                header_row_index = header_rows.index(row)
                break
        
        # 如果仍未找到表头，使用第一行
        if not headers and header_rows:
            header_cells = []
            for cell in header_rows[0].find_all(['th', 'td']):
                cell_text = cell.get_text(strip=True).replace('\n', ' ').replace('\t', ' ').replace('  ', ' ')
                if cell_text:
                    header_cells.append(cell_text)
            headers = header_cells
            print(f"使用第一行作为表头: {headers}")
            header_row_index = 0
        
        # 提取数据行 - 优化数据提取逻辑
        all_rows = table.find_all('tr')
        data_rows = all_rows[header_row_index + 1:] if 'header_row_index' in locals() else all_rows
        
        for row in data_rows:
            row_data = []
            cells = row.find_all(['td', 'th'])
            
            # 跳过空行
            if not cells:
                continue
                
            for cell in cells:
                # 优化文本提取 - 处理多种情况
                cell_text = cell.get_text(strip=True)
                
                # 处理特殊空白字符
                if not cell_text or cell_text in ['\xa0', '&nbsp;']:
                    cell_text = ''
                
                # 清理多余空格
                cell_text = re.sub(r'\s+', ' ', cell_text).strip()
                
                row_data.append(cell_text)
            
            # 确保行数据与表头列数一致
            if row_data:  # 跳过空行
                if len(row_data) != len(headers):
                    # 尝试修复列数不一致的问题
                    if len(row_data) < len(headers):
                        # 补充缺失的列
                        row_data += [''] * (len(headers) - len(row_data))
                    else:
                        # 截断多余的列
                        row_data = row_data[:len(headers)]
                
                # 确保序号列有值
                if headers and len(row_data) > 0 and not row_data[0].strip():
                    # 尝试从其他位置获取序号
                    # 有时序号在第一个单元格的<b>标签中
                    first_cell = cells[0]
                    b_tag = first_cell.find('b')
                    if b_tag:
                        row_data[0] = b_tag.get_text(strip=True)
                
                data.append(row_data)
        
        print(f"获取到 {len(data)} 条记录")
        return headers, data
    except Exception as e:
        print(f"爬取 {province_code} 第 {page} 页失败: {e}")
        traceback.print_exc()
        return headers, data

def scrape_province(province_name, province_code):
    """爬取单个省份的数据"""
    total_pages = get_total_pages(province_code)
    # 限制最大页数为130
    actual_pages = min(total_pages, 130) if total_pages > 0 else 1
    
    print(f"\n开始爬取 {province_name} 数据，共 {actual_pages}/{total_pages} 页...")
    
    all_data = []
    table_headers = []
    
    for page in range(1, actual_pages + 1):
        print(f"\n{'='*30}")
        print(f"处理 {province_name} 第 {page}/{actual_pages} 页")
        print(f"{'='*30}")
        
        headers, page_data = scrape_page(province_code, page)
        
        # 如果是第一页且获取到了表头，保存表头
        if page == 1 and headers:
            table_headers = headers
            print(f"表头: {table_headers}")
        
        # 在每行数据末尾添加省份名称
        for row in page_data:
            row.append(province_name)  # 添加省份列
        
        if page_data:
            all_data.extend(page_data)
            print(f"已爬取第 {page}/{actual_pages} 页，获取 {len(page_data)} 条记录")
            
            # 打印前3条数据
            print("前3条数据样例:")
            for i, row in enumerate(page_data[:3]):
                print(f"{i+1}. {row}")
        else:
            print(f"第 {page} 页无数据")
        
        # 避免请求过快，随机等待1-3秒
        sleep_time = 1 + 2 * random.random()
        print(f"等待 {sleep_time:.2f} 秒后继续...")
        time.sleep(sleep_time)
    
    if all_data:
        # 在表头中添加"省份"列
        if table_headers:
            table_headers.append("省份")
        else:
            # 如果没有表头，创建默认表头
            num_columns = len(all_data[0]) if all_data else 0
            table_headers = [f"列{i+1}" for i in range(num_columns - 1)] + ["省份"]
            print(f"生成默认表头: {table_headers}")
        
        # 保存为CSV文件
        filename = f"rice_data/{province_name.replace('/', '_')}水稻数据.csv"
        with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            # 使用添加了省份列的表头
            writer.writerow(table_headers)
            writer.writerows(all_data)
        print(f"已保存 {len(all_data)} 条数据到 {filename}")
        return len(all_data)
    else:
        print(f"{province_name} 未获取到数据")
        return 0

def main():
    """主函数"""
    print("=" * 50)
    print("国家水稻数据中心数据采集程序")
    print("=" * 50)
    
    # 获取所有省份代码
    provinces = get_province_codes()
    if not provinces:
        print("无法获取省份列表，程序退出")
        return
    
    total_records = 0
    processed_provinces = 0
    total_provinces = len(provinces)
    
    # 爬取所有省份
    for province_name, province_code in provinces.items():
        processed_provinces += 1
        print(f"\n{'#' * 60}")
        print(f"开始处理: {province_name} ({processed_provinces}/{total_provinces})")
        print(f"{'#' * 60}")
        
        try:
            records = scrape_province(province_name, province_code)
            total_records += records
            print(f"已完成 {province_name} 的数据采集，获取 {records} 条记录")
        except Exception as e:
            print(f"处理 {province_name} 时出错: {e}")
            traceback.print_exc()
        
        # 每完成一个省份，等待稍长时间
        sleep_time = 3 + 2 * random.random()
        print(f"等待 {sleep_time:.2f} 秒后处理下一个省份...")
        time.sleep(sleep_time)
    
    print("\n" + "=" * 50)
    print(f"数据采集完成！共获取 {total_records} 条记录")
    print(f"处理了 {processed_provinces} 个省份")
    print("数据保存在 rice_data 文件夹中")
    print("=" * 50)

if __name__ == "__main__":
    main()

国家水稻数据中心数据采集程序
正在获取省份列表...
使用编码: utf-8
找到省份: 农业部 (nation)
找到省份: 浙江 (vejd)
找到省份: 福建 (fujm)
找到省份: 江西 (jdxi)
找到省份: 江　苏 (jdsu)
找到省份: 安徽 (anhv)
找到省份: 上海 (uhhl)
找到省份: 山东 (ujds)
找到省份: 黑龙江 (hlj)
找到省份: 吉林 (jxln)
找到省份: 辽宁 (lcny)
找到省份: 河北 (hebz)
找到省份: 内蒙古 (nmg)
找到省份: 山西 (ujxi)
找到省份: 天津 (tmjn)
找到省份: 北京 (bzjy)
找到省份: 陕　西 (sjxi)
找到省份: 甘肃 (gjsu)
找到省份: 宁夏 (nyxw)
找到省份: 新疆 (xnjd)
找到省份: 湖　南 (hunj)
找到省份: 河南 (henj)
找到省份: 湖北 (hubz)
找到省份: 广　东 (gdds)
找到省份: 广西 (gdxi)
找到省份: 海南 (hlnj)
找到省份: 四　川 (siir)
找到省份: 云南 (ypnj)
找到省份:  (vejd)
找到省份: 贵州 (gvvb)
找到省份: 重庆 (isqy)
共找到 31 个省份/部门

############################################################
开始处理: 农业部 (1/31)
############################################################
获取总页数: https://www.ricedata.cn/variety/identified/nation_1.htm
使用编码: gbk
从末页链接提取总页数: 122

开始爬取 农业部 数据，共 122/122 页...

处理 农业部 第 1/122 页
正在爬取: https://www.ricedata.cn/variety/identified/nation_1.htm
使用编码: gbk
表格HTML片段: <table cellpadding="2" cellspacing="0" style="font-size:13; font-family:Times New Roman;